# Directional Mismatch on the Gardiner
Investigating whether there is a directional mismatch in bluetooth observations, first noticed in comparing traffic data on Blip segments `D_E` and `E_D`, which are the Gardiner from Spadina to Parliament

In [1]:
from psycopg2 import connect
import psycopg2.sql as pg
import configparser
import datetime
%matplotlib inline
import pandas as pd
import pandas.io.sql as pandasql
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set(color_codes=True)
from IPython.display import HTML
def print_table(sql, con):
    return HTML(pandasql.read_sql(sql, con).to_html(index=False))

In [2]:
CONFIG = configparser.ConfigParser()
CONFIG.read('../../db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [4]:
sql = '''SELECT date_trunc('month', datetime_bin) as yyyymm, COUNT(nullif(startpointname, 'E')) AS "Number EB obs", COUNT(nullif(startpointname, 'D')) AS "Number WB obs", COUNT(nullif(startpointname, 'E')) - COUNT(nullif(startpointname, 'D')) AS "EB - WB"
  FROM bluetooth.aggr_5min_i95
  INNER JOIN bluetooth.ref_segments USING (analysis_id)
WHERE startpointname IN ('D','E') AND endpointname IN ('D','E') AND datetime_bin >= '2016-01-01' AND datetime_bin < '2016-01-01'::DATE + INTERVAL '1 Year' 
GROUP BY yyyymm ORDER BY yyyymm;'''
print_table(sql, con)

yyyymm,Number EB obs,Number WB obs,EB - WB
2016-01-01,5444,7272,-1828
2016-02-01,5147,6910,-1763
2016-03-01,5506,7510,-2004
2016-04-01,5080,7100,-2020
2016-05-01,4953,7046,-2093
2016-06-01,5450,7352,-1902
2016-07-01,5961,7732,-1771
2016-08-01,5798,7678,-1880
2016-09-01,5532,7343,-1811
2016-10-01,5003,6880,-1877


In [5]:
sql = '''

SELECT extract('hour' FROM datetime_bin) AS "Hour", COUNT(nullif(startpointname, 'E')) AS "Number EB obs", COUNT(nullif(startpointname, 'D')) AS "Number WB obs", COUNT(nullif(startpointname, 'E')) - COUNT(nullif(startpointname, 'D')) AS "EB - WB"
  FROM bluetooth.aggr_5min_i95
  INNER JOIN bluetooth.ref_segments USING (analysis_id)
WHERE startpointname IN ('D','E') AND endpointname IN ('D','E') AND datetime_bin >= '2016-01-01' AND datetime_bin < '2016-01-01'::DATE + INTERVAL '1 Year' AND EXTRACT('isodow' FROM datetime_bin) < 6
GROUP BY "Hour" ORDER BY "Hour";
'''
print_table(sql, con)

Hour,Number EB obs,Number WB obs,EB - WB
0.0,1063,1601,-538
1.0,637,1031,-394
2.0,407,903,-496
3.0,535,1186,-651
4.0,952,1400,-448
5.0,1892,2601,-709
6.0,2643,3064,-421
7.0,2573,3073,-500
8.0,2467,3085,-618
9.0,2701,3098,-397
